# Quantitative Momentum Investing Strategy
#### For this project, I'm going to build an investing strategy that selects the 50 stocks in the US market with the highest price returns for last 1 year that too with high quality momentum (HQM). From there, an equal-weight portfolio of these 50 stocks will be created.

## Importing libraries

In [1]:
import numpy as np 
import pandas as pd 
import requests 
import math 
from scipy import stats 
from secret1 import IEX_CLOUD_API_TOKEN

In [2]:
import warnings
warnings.filterwarnings('ignore')

## Importing list of stocks in S&P 500 index
#### The constituents of this index change over time, so in an ideal world we would connect directly to the index provider and pull their real-time constituents on a regular basis. But,here I'm using a static version of the S&P 500 constituents (for the purpose of this project) available on web for free & moving this file into the project directory so it can be accessed by other files in the directory.

In [3]:
stocks = pd.read_csv('sp_500_stocks (1).csv')
stocks[:10]

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
5,ABC
6,ABMD
7,ABT
8,ACN
9,ADBE


## Acquiring an API Token
#### Now we need an API Token from one of the data providers to get the financial data. Here I'm using IEX CLOUD API TOKEN and stored my API Token in secret1.py file that is available in project directory.


In [4]:
symbol = 'AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2964509320080,
 'week52high': 199.62,
 'week52low': 140.16,
 'week52highSplitAdjustOnly': 199.62,
 'week52lowSplitAdjustOnly': 141.32,
 'week52change': 0.35186206254679986,
 'sharesOutstanding': 15461896000,
 'float': 0,
 'avg10Volume': 50447086,
 'avg30Volume': 56640183,
 'day200MovingAvg': 183.77,
 'day50MovingAvg': 190.82,
 'employees': 164000,
 'ttmEPS': 6.13,
 'ttmDividendRate': 0.9465536932059656,
 'dividendYield': 0.004936909681353808,
 'nextDividendDate': '',
 'exDividendDate': '2023-11-10',
 'nextEarningsDate': '',
 'peRatio': 30.56352719294809,
 'beta': 1.1345346804441367,
 'maxChangePercent': 73.94136960600376,
 'year5ChangePercent': 4.186040724471469,
 'year2ChangePercent': 0.14297943674297642,
 'year1ChangePercent': 0.3247188263045091,
 'ytdChangePercent': -0.004155196592738886,
 'month6ChangePercent': -0.01570265626291467,
 'month3ChangePercent': 0.14276211916124715,
 'month1ChangePercent': -0.004155196592738886,
 'day30ChangePer

## Creating a Pandas DataFrame for analysis

In [5]:
my_columns = ['Ticker','Name', 'Stock Price ($)', 'One-Year Price Return', 'No. of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Name,Stock Price ($),One-Year Price Return,No. of Shares to Buy


## Using Batch API Calls to Improve Performance & appending data to our DataFrame

In [6]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
   

In [7]:
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()

    for symbol in symbol_string.split(','):
         if symbol in data:
            new_data = {'Ticker':symbol,
                        'Name':data[symbol]['quote']['companyName'],
                        'Stock Price ($)': data[symbol]['quote']['latestPrice'], 
                        'One-Year Price Return':  data[symbol]['stats']['year1ChangePercent'],
                        'No. of Shares to Buy':['NA']}
            new_df = pd.DataFrame(new_data)
           
            final_dataframe = pd.concat([final_dataframe, new_df], ignore_index=True)

final_dataframe 

,Ticker,Name,Stock Price ($),One-Year Price Return,No. of Shares to Buy
0,A,Agilent Technologies Inc.,132.97,-0.140631,NA
1,AAL,American Airlines Group Inc,14.56,-0.090688,NA
2,AAP,Advance Auto Parts Inc,67.79,-0.534363,NA
3,AAPL,Apple Inc,185.85,0.324719,NA
4,ABBV,Abbvie Inc,168.67,0.165845,NA
...,...,...,...,...,...
491,YUM,Yum Brands Inc.,128.75,0.038058,NA
492,ZBH,Zimmer Biomet Holdings Inc,126.32,-0.015937,NA
493,ZBRA,Zebra Technologies Corp. - Class A,246.48,-0.192904,NA
494,ZION,Zions Bancorporation N.A,39.65,-0.115995,NA


## Sorting top 50 High-Momentum Stocks

#### The investment strategy that I'm building seeks to identify the 50 highest-momentum stocks in the S&P 500.Because of this, I need to do remove all the stocks in our DataFrame that fall below this momentum threshold. I'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.

In [8]:
final_dataframe.sort_values('One-Year Price Return', ascending=False, inplace=True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(drop=True, inplace=True)
final_dataframe

,Ticker,Name,Stock Price ($),One-Year Price Return,No. of Shares to Buy
0,LB,L Brands Inc,79.92,2.281287,NA
1,NVDA,NVIDIA Corp,661.60,2.068633,NA
2,AMD,Advanced Micro Devices Inc.,177.66,1.358488,NA
3,ANET,Arista Networks Inc,273.10,1.138505,NA
4,AVGO,Broadcom Inc,1224.34,1.128423,NA
5,PHM,PulteGroup Inc,105.97,1.033846,NA
6,RCL,Royal Caribbean Group,123.44,0.996396,NA
7,LLY,Lilly(Eli) & Co,667.65,0.904010,NA
8,LRCX,Lam Research Corp.,838.70,0.788059,NA
9,CRM,Salesforce Inc,285.66,0.749696,NA


## Calculating the Number of Shares to Buy

In [9]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio: 1000000


1000000


In [10]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'No. of Shares to Buy'] = math.floor(position_size / final_dataframe['Stock Price ($)'][i])
final_dataframe

,Ticker,Name,Stock Price ($),One-Year Price Return,No. of Shares to Buy
0,LB,L Brands Inc,79.92,2.281287,250
1,NVDA,NVIDIA Corp,661.60,2.068633,30
2,AMD,Advanced Micro Devices Inc.,177.66,1.358488,112
3,ANET,Arista Networks Inc,273.10,1.138505,73
4,AVGO,Broadcom Inc,1224.34,1.128423,16
5,PHM,PulteGroup Inc,105.97,1.033846,188
6,RCL,Royal Caribbean Group,123.44,0.996396,162
7,LLY,Lilly(Eli) & Co,667.65,0.904010,29
8,LRCX,Lam Research Corp.,838.70,0.788059,23
9,CRM,Salesforce Inc,285.66,0.749696,70


## Improving the strategy by sorting stocks with High Quality Momentum 
* High-quality momentum stocks show "slow and steady" outperformance over long periods of time.
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards suddenly.

To identify high-quality momentum, I'm going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

In [11]:
my_columns2 = [
                'Ticker', 
                'Name',
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

final_dataframe2 = pd.DataFrame(columns = my_columns2)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        if symbol in data:
            new_data = {'Ticker':symbol, 
                            'Name':data[symbol]['quote']['companyName'],
                            'Price':data[symbol]['quote']['latestPrice'], 
                            'Number of Shares to Buy':'NA', 
                            'One-Year Price Return':data[symbol]['stats']['year1ChangePercent'], 
                            'One-Year Return Percentile':'NA',
                            'Six-Month Price Return':data[symbol]['stats']['month6ChangePercent'],
                            'Six-Month Return Percentile':'NA',
                            'Three-Month Price Return':data[symbol]['stats']['month3ChangePercent'],
                            'Three-Month Return Percentile':'NA',
                            'One-Month Price Return':data[symbol]['stats']['month1ChangePercent'],
                            'One-Month Return Percentile':'NA',
                            'HQM Score':'NA'}
             
            new_df = pd.DataFrame(new_data, index=[0])
            new_df['One-Year Price Return'] = new_df['One-Year Price Return'].fillna(0)
            new_df['Six-Month Price Return'] = new_df['Six-Month Price Return'].fillna(0)
            new_df['Three-Month Price Return'] = new_df['Three-Month Price Return'].fillna(0)
            new_df['One-Month Price Return'] = new_df['One-Month Price Return'].fillna(0)

            final_dataframe2 = pd.concat([final_dataframe2, new_df], ignore_index=True)

final_dataframe2

,Ticker,Name,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,Agilent Technologies Inc.,132.97,NA,-0.140631,NA,0.057698,NA,0.294684,NA,-0.044595,NA,NA
1,AAL,American Airlines Group Inc,14.56,NA,-0.090688,NA,-0.110185,NA,0.368132,NA,0.087336,NA,NA
2,AAP,Advance Auto Parts Inc,67.79,NA,-0.534363,NA,-0.076600,NA,0.347408,NA,0.101682,NA,NA
3,AAPL,Apple Inc,185.85,NA,0.324719,NA,-0.015703,NA,0.142762,NA,-0.004155,NA,NA
4,ABBV,Abbvie Inc,168.67,NA,0.165845,NA,0.107937,NA,0.191075,NA,0.067794,NA,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,YUM,Yum Brands Inc.,128.75,NA,0.038058,NA,-0.037666,NA,0.098187,NA,-0.000842,NA,NA
492,ZBH,Zimmer Biomet Holdings Inc,126.32,NA,-0.015937,NA,-0.114162,NA,0.193577,NA,0.010025,NA,NA
493,ZBRA,Zebra Technologies Corp. - Class A,246.48,NA,-0.192904,NA,-0.144562,NA,0.235303,NA,-0.063659,NA,NA
494,ZION,Zions Bancorporation N.A,39.65,NA,-0.115995,NA,0.189058,NA,0.544204,NA,0.024162,NA,NA


## Calculating Momentum Percentiles

Now I'll calculate momentum percentile scores for every stock in the universe. i.e, I'll calculate percentile scores for the following metrics for every stock:

* One-Year Price Return
* Six-Month Price Return
* Three-Month Price Return
* One-Month Price Return

In [12]:
time_periods = ['One-Year', 'Six-Month', 'Three-Month', 'One-Month']

for row in final_dataframe2.index:
    for time_period in time_periods:
        percentile_col = f'{time_period} Return Percentile'
        price_col = f'{time_period} Price Return'
        final_dataframe2.loc[row, percentile_col] = stats.percentileofscore(final_dataframe2[price_col], final_dataframe2.loc[row, price_col])

final_dataframe2

,Ticker,Name,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,Agilent Technologies Inc.,132.97,NA,-0.140631,14.516129,0.057698,57.862903,0.294684,77.217742,-0.044595,15.120968,NA
1,AAL,American Airlines Group Inc,14.56,NA,-0.090688,23.58871,-0.110185,14.919355,0.368132,89.516129,0.087336,93.75,NA
2,AAP,Advance Auto Parts Inc,67.79,NA,-0.534363,0.806452,-0.076600,20.16129,0.347408,86.693548,0.101682,95.766129,NA
3,AAPL,Apple Inc,185.85,NA,0.324719,82.66129,-0.015703,37.701613,0.142762,37.701613,-0.004155,39.112903,NA
4,ABBV,Abbvie Inc,168.67,NA,0.165845,65.322581,0.107937,67.741935,0.191075,52.620968,0.067794,88.306452,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,YUM,Yum Brands Inc.,128.75,NA,0.038058,50.0,-0.037666,30.241935,0.098187,26.612903,-0.000842,42.943548,NA
492,ZBH,Zimmer Biomet Holdings Inc,126.32,NA,-0.015937,37.298387,-0.114162,14.314516,0.193577,53.225806,0.010025,54.435484,NA
493,ZBRA,Zebra Technologies Corp. - Class A,246.48,NA,-0.192904,9.072581,-0.144562,9.677419,0.235303,64.314516,-0.063659,8.669355,NA
494,ZION,Zions Bancorporation N.A,39.65,NA,-0.115995,18.346774,0.189058,83.064516,0.544204,98.58871,0.024162,64.314516,NA


## Calculating the HQM Score

I'll now calculate HQM Score, which is the high-quality momentum score that I'll use to filter for stocks in this investing strategy.

The HQM Score will be the arithmetic mean of the 4 momentum percentile scores that I calculated earlier.

In [13]:
from statistics import mean

for row in final_dataframe2.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(final_dataframe2.loc[row, f'{time_period} Return Percentile'])
    final_dataframe2.loc[row, 'HQM Score'] = mean(momentum_percentiles)

final_dataframe2

,Ticker,Name,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,Agilent Technologies Inc.,132.97,NA,-0.140631,14.516129,0.057698,57.862903,0.294684,77.217742,-0.044595,15.120968,41.179435
1,AAL,American Airlines Group Inc,14.56,NA,-0.090688,23.58871,-0.110185,14.919355,0.368132,89.516129,0.087336,93.75,55.443548
2,AAP,Advance Auto Parts Inc,67.79,NA,-0.534363,0.806452,-0.076600,20.16129,0.347408,86.693548,0.101682,95.766129,50.856855
3,AAPL,Apple Inc,185.85,NA,0.324719,82.66129,-0.015703,37.701613,0.142762,37.701613,-0.004155,39.112903,49.294355
4,ABBV,Abbvie Inc,168.67,NA,0.165845,65.322581,0.107937,67.741935,0.191075,52.620968,0.067794,88.306452,68.497984
...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,YUM,Yum Brands Inc.,128.75,NA,0.038058,50.0,-0.037666,30.241935,0.098187,26.612903,-0.000842,42.943548,37.449597
492,ZBH,Zimmer Biomet Holdings Inc,126.32,NA,-0.015937,37.298387,-0.114162,14.314516,0.193577,53.225806,0.010025,54.435484,39.818548
493,ZBRA,Zebra Technologies Corp. - Class A,246.48,NA,-0.192904,9.072581,-0.144562,9.677419,0.235303,64.314516,-0.063659,8.669355,22.933468
494,ZION,Zions Bancorporation N.A,39.65,NA,-0.115995,18.346774,0.189058,83.064516,0.544204,98.58871,0.024162,64.314516,66.078629


## Selecting the 50 High Quality Momentum Stocks

In [14]:
final_dataframe2.sort_values('HQM Score', ascending=False, inplace=True)
final_dataframe2 = final_dataframe2[:50]
final_dataframe2.reset_index(drop=True, inplace=True)
final_dataframe2

,Ticker,Name,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,AMD,Advanced Micro Devices Inc.,177.66,NA,1.358488,99.596774,0.574274,99.395161,0.844136,100.0,0.206363,99.193548,99.546371
1,ANET,Arista Networks Inc,273.10,NA,1.138505,99.395161,0.784235,99.596774,0.530053,97.379032,0.144665,98.58871,98.739919
2,NVDA,NVIDIA Corp,661.60,NA,2.068633,99.798387,0.336377,95.967742,0.542481,98.387097,0.261359,99.596774,98.4375
3,URI,"United Rentals, Inc.",654.20,NA,0.513259,93.548387,0.461444,98.991935,0.628799,98.991935,0.129591,97.580645,97.278226
4,NFLX,Netflix Inc.,564.64,NA,0.596003,96.975806,0.352318,96.572581,0.447181,94.959677,0.182612,98.991935,96.875
5,AVGO,Broadcom Inc,1224.34,NA,1.128423,99.193548,0.373303,97.177419,0.465947,95.967742,0.090947,94.758065,96.774194
6,NOW,ServiceNow Inc,781.30,NA,0.715643,97.983871,0.382238,97.379032,0.420985,92.943548,0.114297,96.774194,96.270161
7,CRM,Salesforce Inc,285.66,NA,0.749696,98.185484,0.275975,91.733871,0.464415,95.766129,0.093942,94.959677,95.16129
8,WDC,Western Digital Corp.,58.16,NA,0.328664,83.266129,0.421604,97.983871,0.533231,97.782258,0.140920,98.185484,94.304435
9,IBM,International Business Machines Corp.,185.79,NA,0.461674,90.927419,0.334545,95.766129,0.327971,83.669355,0.144237,98.387097,92.1875


## Calculating the Number of Shares to Buy

In [15]:
portfolio_input()

position_size = float(portfolio_size)/len(final_dataframe2)

for i in final_dataframe2.index:
    final_dataframe2.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe2['Price'][i])

final_dataframe2

Enter the value of your portfolio: 1000000


,Ticker,Name,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,AMD,Advanced Micro Devices Inc.,177.66,112,1.358488,99.596774,0.574274,99.395161,0.844136,100.0,0.206363,99.193548,99.546371
1,ANET,Arista Networks Inc,273.10,73,1.138505,99.395161,0.784235,99.596774,0.530053,97.379032,0.144665,98.58871,98.739919
2,NVDA,NVIDIA Corp,661.60,30,2.068633,99.798387,0.336377,95.967742,0.542481,98.387097,0.261359,99.596774,98.4375
3,URI,"United Rentals, Inc.",654.20,30,0.513259,93.548387,0.461444,98.991935,0.628799,98.991935,0.129591,97.580645,97.278226
4,NFLX,Netflix Inc.,564.64,35,0.596003,96.975806,0.352318,96.572581,0.447181,94.959677,0.182612,98.991935,96.875
5,AVGO,Broadcom Inc,1224.34,16,1.128423,99.193548,0.373303,97.177419,0.465947,95.967742,0.090947,94.758065,96.774194
6,NOW,ServiceNow Inc,781.30,25,0.715643,97.983871,0.382238,97.379032,0.420985,92.943548,0.114297,96.774194,96.270161
7,CRM,Salesforce Inc,285.66,70,0.749696,98.185484,0.275975,91.733871,0.464415,95.766129,0.093942,94.959677,95.16129
8,WDC,Western Digital Corp.,58.16,343,0.328664,83.266129,0.421604,97.983871,0.533231,97.782258,0.140920,98.185484,94.304435
9,IBM,International Business Machines Corp.,185.79,107,0.461674,90.927419,0.334545,95.766129,0.327971,83.669355,0.144237,98.387097,92.1875


## Saving Our Pandas DataFrame to Excel Output

In [16]:
excel_file_path = 'momentum_investing_fund.xlsx'

# Save the DataFrame to Excel
final_dataframe2.to_excel(excel_file_path, index=False)